In [1]:
import os

### openBLAS will not work properly in jupyter notebook
os.environ['OPENBLAS_NUM_THREADS'] = '1'

import pandas as pd
import numpy as np
import anndata as ad
from scipy import stats as ss
import snapatac2 as snap
import scanpy as sc
import mudata
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import colors

## Read and calculate cell type rss

In [2]:
outdir = "/projects/ps-renlab2/y2xie/projects/BICAN/analysis/11.GRN/scplus_pipeline/Snakemake/"
scplus_mdata = mudata.read(f"{outdir}/scplusmdata.h5mu")
scplus_mdata

/home/y2xie/miniconda3/envs/seurat/lib/python3.11/site-packages/mudata/_core/mudata.py:1598: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/home/y2xie/miniconda3/envs/seurat/lib/python3.11/site-packages/mudata/_core/mudata.py:1461: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


MuData object with n_obs × n_vars = 116373 × 544466
  uns:	'direct_e_regulon_metadata', 'extended_e_regulon_metadata'
  6 modalities
    scRNA_counts:	116373 x 38369
      obs:	'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'percent.ribo', 'doublet_score', 'doublet_ident', 'dna', 'rna', 'region', 'structure', 'donor', 'target', 'rna_bc', 'atac_bc', 'index', 'region.color', 'region.name', 'structure.color', 'label', 'prob', 'RNA_snn_res.0.5', 'seurat_clusters', 'filter', 'cluster', 'subcluster', 'supercluster_name', 'supercluster_bootstrapping_probability', 'cluster_name', 'cluster_bootstrapping_probability', 'subcluster_name', 'subcluster_bootstrapping_probability', 'subclass', 'subclass_corrected', 'keep', 'cluster_manual_name', 'cluster_manual', 'supercluster_manual', 'class', 'final_rna_bc'
      var:	'vf_vst_counts.1_mean', 'vf_vst_counts.1_variance', 'vf_vst_counts.1_variance.expected', 'vf_vst_counts.1_variance.standardized', 'vf_vst_counts.1_variable', 'vf_vst_counts.1_rank', 'vf_vst_counts.2_mean', 'vf_vst_counts.2_variance', 'vf_vst_counts.2_variance.expected', 'vf_vst_counts.2_variance.standardized', 'vf_vst_counts.2_variable', 'vf_vst_counts.2_rank', 'var.features', 'var.features.rank'
      obsm:	'X_harmony', 'X_pca', 'X_umap'
    scATAC_counts:	116373 x 505317
      obs:	'cisTopic_nr_frag', 'cisTopic_log_nr_frag', 'cisTopic_nr_acc', 'cisTopic_log_nr_acc', 'sample_id', 'n_fragment', 'frac_dup', 'frac_mito', 'doublet_probability', 'doublet_score_x', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'percent.ribo', 'doublet_score_y', 'doublet_ident', 'dna', 'rna', 'region', 'structure', 'donor', 'target', 'rna_bc', 'atac_bc', 'index', 'condition', 'leiden', 'subclass', 'subclass_corrected', 'supercluster_manual', 'cluster_manual_name'
      var:	'Chromosome', 'Start', 'End', 'Width', 'cisTopic_nr_frag', 'cisTopic_log_nr_frag', 'cisTopic_nr_acc', 'cisTopic_log_nr_acc'
      obsm:	'UMAP'
    direct_gene_based_AUC:	116373 x 249
    direct_region_based_AUC:	116373 x 249
    extended_gene_based_AUC:	116373 x 141
    extended_region_based_AUC:	116373 x 141

In [4]:
obj = ad.AnnData(scplus_mdata["direct_gene_based_AUC"])
obj.write(f"{outdir}/direct_gene_based_AUC.h5ad")

obj = ad.AnnData(scplus_mdata["direct_region_based_AUC"])
obj.write(f"{outdir}/direct_region_based_AUC.h5ad")

In [3]:
from scenicplus.RSS import (regulon_specificity_scores, plot_rss)
rss = regulon_specificity_scores(
    scplus_mudata = scplus_mdata,
    variable = "scRNA_counts:subclass_corrected",
    modalities = ["direct_gene_based_AUC"]
)

rss.T.to_csv("subclass_corrected.RSS.csv")

In [4]:
rss = regulon_specificity_scores(
    scplus_mudata = scplus_mdata,
    variable = "scRNA_counts:cluster_manual_name",
    modalities = ["direct_gene_based_AUC"]
)
rss.T.to_csv("cluster_manual_name.RSS.csv")

## Process H3K27me3 scenic+ results

In [2]:
outdir = "/projects/ps-renlab2/y2xie/projects/BICAN/analysis/11.GRN/scplus_me3_pipeline/Snakemake/workflow/"
scplus_mdata = mudata.read(f"{outdir}/scplusmdata.h5mu")
scplus_mdata

/home/y2xie/miniconda3/envs/seurat/lib/python3.11/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/y2xie/miniconda3/envs/seurat/lib/python3.11/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/y2xie/miniconda3/envs/seurat/lib/python3.11/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/y2xie/miniconda3/envs/seurat/lib/python3.11/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/y2xie/miniconda3/envs/seurat/lib/python3.11/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/y2xie/miniconda3/envs/seurat/

MuData object with n_obs × n_vars = 111343 × 503242
  uns:	'direct_e_regulon_metadata', 'extended_e_regulon_metadata'
  6 modalities
    scRNA_counts:	111343 x 38369
      obs:	'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'percent.ribo', 'doublet_score', 'doublet_ident', 'dna', 'rna', 'region', 'structure', 'donor', 'target', 'rna_bc', 'atac_bc', 'index', 'region.color', 'region.name', 'structure.color', 'label', 'prob', 'RNA_snn_res.0.5', 'seurat_clusters', 'filter', 'cluster', 'subcluster', 'supercluster_name', 'supercluster_bootstrapping_probability', 'cluster_name', 'cluster_bootstrapping_probability', 'subcluster_name', 'subcluster_bootstrapping_probability', 'subclass', 'subclass_corrected', 'keep', 'cluster_manual_name', 'cluster_manual', 'supercluster_manual', 'class', 'nCount_DORC_ac', 'nFeature_DORC_ac', 'nCount_DORC_me3', 'nFeature_DORC_me3', 'final_rna_bc'
      var:	'vf_vst_counts.1_mean', 'vf_vst_counts.1_variance', 'vf_vst_counts.1_variance.expected', 'vf_vst_counts.1_variance.standardized', 'vf_vst_counts.1_variable', 'vf_vst_counts.1_rank', 'vf_vst_counts.2_mean', 'vf_vst_counts.2_variance', 'vf_vst_counts.2_variance.expected', 'vf_vst_counts.2_variance.standardized', 'vf_vst_counts.2_variable', 'vf_vst_counts.2_rank', 'var.features', 'var.features.rank'
      obsm:	'X_harmony', 'X_pca', 'X_umap'
    scATAC_counts:	111343 x 464515
      obs:	'cisTopic_nr_frag', 'cisTopic_log_nr_frag', 'cisTopic_nr_acc', 'cisTopic_log_nr_acc', 'sample_id', 'n_fragment', 'frac_dup', 'frac_mito', 'doublet_probability', 'doublet_score_x', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'percent.ribo', 'doublet_score_y', 'doublet_ident', 'dna', 'rna', 'region', 'structure', 'donor', 'target', 'rna_bc', 'atac_bc', 'index', 'condition', 'leiden', 'subclass', 'subclass_corrected', 'supercluster_manual', 'cluster_manual_name', 'class'
      var:	'Chromosome', 'Start', 'End', 'Width', 'cisTopic_nr_frag', 'cisTopic_log_nr_frag', 'cisTopic_nr_acc', 'cisTopic_log_nr_acc'
    direct_gene_based_AUC:	111343 x 125
    direct_region_based_AUC:	111343 x 125
    extended_gene_based_AUC:	111343 x 54
    extended_region_based_AUC:	111343 x 54

In [6]:
obj = ad.AnnData(scplus_mdata["direct_gene_based_AUC"])
obj.write(f"{outdir}/direct_gene_based_AUC.h5ad")

obj = ad.AnnData(scplus_mdata["direct_region_based_AUC"])
obj.write(f"{outdir}/direct_region_based_AUC.h5ad")

In [3]:
### For H3K27me3: target genes should be repressed
from scenicplus.RSS import (regulon_specificity_scores, plot_rss)
rss = regulon_specificity_scores(
    scplus_mudata = scplus_mdata,
    variable = "scRNA_counts:subclass_corrected",
    modalities = ["direct_region_based_AUC"]
)

rss.T.to_csv("me3_subclass_corrected.region.RSS.csv")

In [5]:
rss = regulon_specificity_scores(
    scplus_mudata = scplus_mdata,
    variable = "scRNA_counts:cluster_manual_name",
    modalities = ["direct_region_based_AUC"]
)
rss.T.to_csv("me3_cluster_manual_name.region.RSS.csv")